In [4]:
!pip install fuzzywuzzy

In [2]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.


**Importing** **Library**

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf


from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pathlib import Path
from google.colab import files

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from google.colab import drive
import nltk
import joblib
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

from tensorflow.keras.models import load_model
import joblib

ModuleNotFoundError: No module named 'tensorflow'

**Dataset**

In [ ]:
# Load dataset

file_path = 'beasiswa.csv'
beasiswa = pd.read_csv(file_path)

In [ ]:
beasiswa.head()

In [ ]:
beasiswa.info()

In [ ]:
file_path = 'beasiswa.csv'

try:
    # Membaca file CSV menjadi DataFrame
    beasiswa = pd.read_csv(file_path)

    # Memisahkan fitur (X) dan target (y)
    X = beasiswa[['Jenjang', 'Jenis Pendanaan', 'Benua']]
    y = beasiswa['Nama Beasiswa']

    # Menentukan proporsi data untuk set pengujian (misalnya, 20%)
    test_size = 0.2
    total_data = len(beasiswa)
    split_index = int(total_data * (1 - test_size))

    # Membagi data menjadi set pelatihan dan pengujian
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    # Contoh print untuk memeriksa ukuran set pelatihan dan pengujian
    print("Jumlah data dalam set pelatihan (X_train):", len(X_train))
    print("Jumlah data dalam set pengujian (X_test):", len(X_test))

except FileNotFoundError:
    print(f"File '{file_path}' not found. Please check the file path.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# Fungsi preprocessing yang diperbarui
def preprocessing(data):
    if isinstance(data, str):
        data = data.lower()
        # Tambahkan langkah-langkah preprocessing lainnya di sini
    return data

**Preference**

In [ ]:
data_beasiswa = beasiswa.copy()

for index, row in data_beasiswa.iterrows():
    # Kategorisasi Biaya
    if row['Jenis Pendanaan'] == 'Fully Funded':
        data_beasiswa.at[index, 'Jenis Pendanaan Tags'] = 'Fully Funded'
    else:
        data_beasiswa.at[index, 'Jenis Pendanaan Tags'] = 'Partially Funded'


# Preprocess the data
data_beasiswa = data_beasiswa.fillna('')

# Penggabungan Tags
data_beasiswa['Tags'] = data_beasiswa['Jenis Pendanaan'] + ' ' + data_beasiswa['Jenjang']+ ' ' + data_beasiswa['Benua']

# Hapus kolom yang tidak diperlukan
data_beasiswa

In [ ]:
data_beasiswa.Tags = data_beasiswa.Tags.apply(preprocessing)
data_beasiswa

In [ ]:
# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data_beasiswa['Tags'])

# Convert the TF-IDF matrix to an array
tfidf_array = tfidf_matrix.toarray()

def preprocess_query(query):
    tokens = word_tokenize(query)
    filtered_query = ' '.join([word.lower() for word in tokens if word.lower() not in ENGLISH_STOP_WORDS and word.lower() not in stopwords.words('indonesian')])
    return filtered_query

# Initialize the recommendation function
def recommend_scholarships(query, top_n=5):
    processed_query = preprocess_query(query)
    query_vector = tfidf_vectorizer.transform([processed_query])

    # Calculate cosine similarity between the query and all scholarships
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()

    # Get indices of top_n most similar scholarships
    indices = cosine_similarities.argsort()[:-top_n-1:-1]

    # Extract recommended scholarship names based on indices
    recommended_scholarships = data_beasiswa.iloc[indices]['Nama Beasiswa'].tolist()
    return recommended_scholarships

In [ ]:
# Get user input and display recommendations
def get_user_input():
    jenis_pendanaan = input("Masukkan Jenis Pendanaan: ")
    jenjang = input("Masukkan Jenjang: ")
    benua = input("Masukkan Benua: ")
    query = f"{jenis_pendanaan} {jenjang} {benua}"
    recommendations = recommend_scholarships(query)
    print("Recommended scholarships:")
    for scholarship in recommendations:
        print(scholarship)

# Call the get_user_input function
get_user_input()

In [ ]:
def preprocess_query(query):
    stop_words_english = set(stopwords.words('english'))
    stop_words_indonesian = set(stopwords.words('indonesian'))

    tokens = word_tokenize(query)

    filtered_query = ' '.join([word.lower() for word in tokens if word.lower() not in ENGLISH_STOP_WORDS and word.lower() not in stopwords.words('indonesian')])

    return filtered_query

# Example usage:
query = 's1 di Amerika Selatan'
preprocessed_query = preprocess_query(query)
recommendations = recommend_scholarships(preprocessed_query)

print("Recommended scholarships:")
for scholarship in recommendations:
    print(scholarship)

**Searching (Content Based Filtering)**

In [ ]:
data_content_based_filtering = beasiswa.copy()

data_content_based_filtering['Tags']  = data_content_based_filtering['Jenis Pendanaan']+' ' + data_content_based_filtering['Jenjang'] + ' ' + data_content_based_filtering['Benua']
data_content_based_filtering

In [ ]:
data_content_based_filtering.Tags = data_content_based_filtering.Tags.apply(preprocessing)
data_content_based_filtering

In [ ]:
def preprocess_query(query):
    stop_words_english = set(stopwords.words('english'))
    stop_words_indonesian = set(stopwords.words('indonesian'))

    tokens = word_tokenize(query)

    filtered_query = ' '.join([word.lower() for word in tokens if word.lower() not in ENGLISH_STOP_WORDS and word.lower() not in stopwords.words('indonesian')])

    return filtered_query

def recommend_by_content_based_filtering(query):
    query = preprocess_query(query)
    matched_beasiswas = []

    for index, beasiswa in data_content_based_filtering.iterrows():
        jenjang_similarity = fuzz.token_set_ratio(query, str(beasiswa['Jenjang']).lower())

        # Give higher weight to 'Region' term
        if isinstance(beasiswa['Benua'], str):
            benua_similarity = 1.5 * fuzz.token_set_ratio(query, beasiswa['Benua'].lower())
        else:
            benua_similarity = 0

        if isinstance(beasiswa['Tags'], str):
            tags_similarity = fuzz.token_set_ratio(query, beasiswa['Tags'].lower())
        else:
            tags_similarity = 0

        combined_similarity = (jenjang_similarity + benua_similarity + tags_similarity) / 3
        matched_beasiswas.append((index, combined_similarity))

    matched_beasiswas = sorted(matched_beasiswas, key=lambda x: x[1], reverse=True)
    matched_beasiswas = matched_beasiswas[:10]

    recommended_nama_beasiswas = []
    for match in matched_beasiswas:
        beasiswa_index = match[0]
        beasiswa_name = data_content_based_filtering.iloc[beasiswa_index]['Nama Beasiswa']
        recommended_nama_beasiswas.append(beasiswa_name)

    return recommended_nama_beasiswas

In [ ]:
def get_scholarship_details(scholarship_name):
    scholarship_details = data_content_based_filtering[data_content_based_filtering['Nama Beasiswa'] == scholarship_name]

    print("Scholarship Details:")
    print(scholarship_details)

In [ ]:
recommendations = recommend_by_content_based_filtering('S3 di Eropa')
print("Recommended scholarships:")
for scholarship in recommendations:
    print(scholarship)
    get_scholarship_details(scholarship)

In [ ]:
# Menyimpan matriks TF-IDF
joblib.dump(tfidf_matrix, 'tfidf_matrix.pkl')

# Menyimpan objek TfidfVectorizer (opsional)
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')

In [ ]:
!ls

In [ ]:
files.download('tfidf_matrix.pkl')